# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
    * [Connect to Workspace](#Connect-to-Workspace)
* [Data](#Data)
    * [Load Data](#Load-Data)
* [Model](#Model)
    * [Retrieve Model](#Retrieve-Model)
    * [Download Model](#Download-Model)
    * [Load Model](#Load-Model)
    * [Model Evaluation](#Model-Evaluation)
---

# Notebook Summary

This notebook will evaluate the model trained in `02_model_training` on the stanford dog test set and add the the calculated test accuracy to the properties of the registered Azure Machine Learning (AML) model.

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [1]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

Automatically reload modules when changes are made.

In [2]:
%load_ext autoreload
%autoreload 2

Import libraries and modules.

In [ ]:
# Import libraries
import azureml.core
import numpy as np
import os
import torch
from azureml.core import Model, Workspace

# Import created modules
from src.utils import load_data

print(f"azureml.core version: {azureml.core.VERSION}")

### Connect to Workspace

In order to connect and communicate with the AML workspace, a workspace object needs to be instantiated using the AML SDK.

In [4]:
# Connect to the AML workspace using interactive authentication
ws = Workspace.from_config()

# Data

### Load Data

Use the previously created utility function to create dataloaders and retrieve dataset_sizes.

In [5]:
# Load data
dataloaders, dataset_sizes, class_names = load_data("../data")

# Model

### Retrieve Model

In [8]:
aml_model = Model(ws, name="dog-classification-model")

### Download Model

In [ ]:
# Create directory
outputs_folder = os.path.join(os.getcwd(), "../outputs")
os.makedirs(outputs_folder, exist_ok=True)
print(f"Outputs folder {outputs_folder} has been created.")

# Download model artifact
aml_model.download(target_dir="../outputs/dog_clf_model.pt")

### Load Model

Load the model that was trained in the `02_model_training` notebook.

In [ ]:
model_path = "../outputs/dog_clf_model.pt"

model = torch.load(model_path, map_location=lambda storage, loc: storage)
model.eval()

### Model Evaluation

Evaluate the model on the test set by calculating the accuracy (which is our primary evaluation metric).

In [ ]:
# Leverage GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize running_correct_preds (this will later be updated with every batch)
running_correct_preds = 0

# Use the test dataloader to load in data in batches
for inputs, labels in dataloaders["test"]:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Make predictions on test data batch
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
    # Add correct predictions from current batch
    running_correct_preds += torch.sum(preds == labels.data)

In [ ]:
# Calculate the total accuracy over the test set
test_accuracy = running_correct_preds.double() / dataset_sizes["test"]
print(f"Test accuracy: {np.round(test_accuracy.numpy(), 4)*100}%")

Add test accuracy to model properties.

In [ ]:
# Add test accuracy to model properties
aml_model.add_properties({"test_acc": str(np.round(test_accuracy.numpy(), 4)*100)})